In [3]:
import geopandas as gpd
import rasterio 
import os
import pandas as pd

os.chdir("/home/silas/projects/msc_thesis")

In [41]:
#set Api key for ORS
api_key= "5b3ce3597851110001cf624865e19fb4d0c2400e9aba8877785f6853"

#import flat dataset
flats_zh= gpd.read_file(r'./data/raw_data/geodata_stadt_Zuerich/building_stats/data/ssz.gwr_stzh_wohnungen.shp')

#import rcp dataset
rcps= gpd.read_file(r'./data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')





In [42]:

flats_zh['wstat'] = flats_zh['wstat'].astype('str')

print(flats_zh['wstatlang'].unique())



['Bestehend' 'Aufgehoben' 'Nicht realisiert' 'Bewilligt' 'Im Bau'
 'Freigegeben']


In [ ]:
#filter out demolished flats
flats_zh_existing=flats_zh.query('wstatlang=="Bestehend"')
len(flats_zh_existing)

# check if CRS are matching
print("Current CRS:", flats_zh_existing.crs)
print("Current CRS:", rcps.crs)



240531

In [68]:
#create a subset of 1000 flats
n=10000
buffer_distance=600
flats_subset=flats_zh_existing.iloc[1:n,]

flats_subset['buffer'] = flats_subset.geometry.buffer(buffer_distance)

def find_points_in_buffer(buffer_geom):
    return rcps[rcps.geometry.within(buffer_geom)]


#apply function to each buffer
results = []
for idx, row in flats_subset.iterrows():
    points_in_buffer = find_points_in_buffer(row['buffer'])
    if not points_in_buffer.empty:
        results.append({'flat_id': idx,
                        'recycling_points': points_in_buffer.index.tolist()
                        })
results

/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


[{'flat_id': 1, 'recycling_points': [47, 48, 129, 130, 131]},
 {'flat_id': 2, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 3, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 4, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 5, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 6, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 7, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 8, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 9, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 10, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 11, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 12, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 13, 'recycling_points': [48, 129, 130, 131]},
 {'flat_id': 14, 'recycling_points': [47, 48, 78, 130, 131]},
 {'flat_id': 15, 'recycling_points': [47, 48, 78, 130, 131]},
 {'flat_id': 16, 'recycling_points': [47, 48, 78, 130, 131]},
 {'flat_id': 17, 'recycling_points': [47, 48, 78, 13

1       POLYGON ((2682865.535 1246779.539, 2682858.312...
2       POLYGON ((2682776.647 1246810.577, 2682769.424...
3       POLYGON ((2682771.061 1246805.713, 2682763.838...
4       POLYGON ((2682748.337 1246775.161, 2682741.114...
5       POLYGON ((2682793.063 1246825.017, 2682785.840...
                              ...                        
1063    POLYGON ((2685413.341 1246655.105, 2685406.118...
1064    POLYGON ((2685420.367 1246644.776, 2685413.144...
1065    POLYGON ((2685420.367 1246644.776, 2685413.144...
1066    POLYGON ((2685355.762 1246640.144, 2685348.539...
1067    POLYGON ((2685355.762 1246640.144, 2685348.539...
Name: buffer, Length: 999, dtype: geometry